## Riddler Classic: 

Duane’s friend’s granddaughter claimed that she once won a game of War that lasted exactly 26 turns.

War is a two-player game in which a standard deck of cards is first shuffled and then divided into two piles with 26 cards each — one pile for each player. In every turn of the game, both players flip over and reveal the top card of their deck. The player whose card has a higher rank wins the turn and places both cards on the bottom of their pile. In the event that both cards have the same rank, the rules get a little more complicated, with each player flipping over additional cards to compare in a mini “War” showdown. Duane’s friend’s granddaughter said that for every turn of the game, she always drew the card of higher rank, with no mini “Wars.”

Assuming a deck is randomly shuffled before every game, how many games of War would you expect to play until you had a game that lasted just 26 turns with no “Wars,” like Duane’s friend’s granddaughter?

### Python Simulation:

- Build a basic deck and divide up
    - Just using integers here: 
        - 2:10 are normal
        - J, Q, K, A -> 11-14
        - no joker
    - need to attach a suit, this is where named tuple is helpful
    
- Build the War class

- Run many iterations (possibly use multiprocessing to store results?)

In [34]:
import collections
import random

# a card has two components: rank & suit 
# named tuples are easy-to-create, lightweight object types
Card = collections.namedtuple('Card', ['rank', 'suit'])

# similar idea to CardDeck class built from Fluent Python Ch 1
class CardDeck:
    ranks = [i for i in range(2,15)] # lazy way of doing this 
    suits = 's d c h'.split() 
    
    def __init__(self):
        self.cards = [Card(rank, suit) for suit in self.suits 
                      for rank in self.ranks]
        
    def deal(self):
        """Shuffle cards & deal out in batches of 26"""
        
        self.__shuffle() # shuffle cards
        
        return self.cards[:26], self.cards[26:]
    
    def __shuffle(self):
        """Randomly order list of cards to simulate shuffling"""
        random.shuffle(self.cards)

In [38]:
# run a basic test of dealing
test = CardDeck()
p1, p2 = test.deal()
print(f"P1 gets {len(p1)} cards; P2 gets {len(p2)} cards")

P1 gets 26 cards; P2 gets 26 cards


### War Game: 

General Rules:
- each player flips top of deck (index 0)
- player that wins receives card:
    - winning player will move played card & winning card to bottom of stack (maybe randomize order???)
- in case of tie:
    - if rank is equivalent, then two new cards are dealt.
    - one card is face-down, and one is face-up
    - 
    
Making it a bit easier to simulate: 

- limit processing:
    - Given the challenge, we really just want to know the number of times our iteration was exactly 26. So we can really just play this to a binary end:
       - If the first person to win a card then loses, mark as '0'
       - If a tie ever occurs mark as '0'
       - If first person to win always wins until one deck is empty, mark as '1'

- Going to pivot here. We don't actually need the total card deck above. We just need numbers, and need to check if one sequence always exceeds another ssequence element by element
- numpy should be faster
        

### Pivoting: Simpler version of this

Realized while coding the card game that we really only care about if one sequence has all elements > another sequence

This is quick to solve in nympy as I can subtract arrays from one another. Then just need to check if every element is positive or negative:
- if yes then game is over and receives a `1`
- if no then game would continue beyond 26 plays and receives `0`

In [81]:
import numpy as np

class leanDeck:
    def __init__(self):
        self.deck = np.tile(np.arange(13),4) # build 52 total integers of 1-13
        
    def play_war(self):
        """We can play a simple game of war that tracks automatic win vs non-automatic"""
        
        # shuffle & play
        self.__shuffle()
        
        # find array of differences
        diff = self.p1 - self.p2
        
        # check logic of all elements - either must all be positive or negative
        if (diff > 0).all():
            return 1
        elif (diff < 0).all():
            return 1
        else:
            return 0
        
                
        
    def __shuffle(self):
        """in place shufflinf to simulate shuffling deck"""
        np.random.shuffle(self.deck)
        
        self.p1 = self.deck[:26] # first 26 goes to p1
        self.p2 = self.deck[26:] # second 26 goes to p2
            

In [88]:
tracker = 0 # used to store number of times we see the result hoped for 
tot_its = 0

for _ in range(10000000):
    
    # generate new class and play
    simGame = leanDeck()
    tracker += simGame.play_war() # update tracker with result 
    tot_its += 1 # update iterations

In [90]:
tracker

0

### rebuild using numba....

numba lets u use the JIT compiler, so should be faster if written properly 

In [94]:
from numba import jit

In [95]:
x = np.arange(100).reshape(10, 10)

@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
def go_fast(a): # Function is compiled to machine code when called the first time
    trace = 0.0
    for i in range(a.shape[0]):   # Numba likes loops
        trace += np.tanh(a[i, i]) # Numba likes NumPy functions
    return a + trace              # Numba likes NumPy broadcasting

print(go_fast(x))

[[  9.  10.  11.  12.  13.  14.  15.  16.  17.  18.]
 [ 19.  20.  21.  22.  23.  24.  25.  26.  27.  28.]
 [ 29.  30.  31.  32.  33.  34.  35.  36.  37.  38.]
 [ 39.  40.  41.  42.  43.  44.  45.  46.  47.  48.]
 [ 49.  50.  51.  52.  53.  54.  55.  56.  57.  58.]
 [ 59.  60.  61.  62.  63.  64.  65.  66.  67.  68.]
 [ 69.  70.  71.  72.  73.  74.  75.  76.  77.  78.]
 [ 79.  80.  81.  82.  83.  84.  85.  86.  87.  88.]
 [ 89.  90.  91.  92.  93.  94.  95.  96.  97.  98.]
 [ 99. 100. 101. 102. 103. 104. 105. 106. 107. 108.]]


In [113]:
deck = np.tile(np.arange(13),4) # build 52 total integers of 0-12

@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
def war_game(deck): # Function is compiled to machine code when called the first time
    np.random.shuffle(deck) # shuffle 
    p1 = deck[:26]
    p2 = deck[26:]
    
    # find array of differences
    diff = p1 - p2
    
    # check logic of all elements - either must all be positive or negative
    if (diff > 0).all():
        return 1
    elif (diff < 0).all():
        return 1
    else:
        return 0

### time test:

see how numba performs compared to class approach - looks like its about 15X faster

In [116]:
%timeit war_game(deck)

1.5 µs ± 71.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [117]:
def no_numba():
    # generate new class and play
    simGame = leanDeck()
    return simGame.play_war()


In [118]:
%timeit no_numba()

20.9 µs ± 977 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### run with numba 100 million times?

- the deck is just getting perpetually shuffled....shouldn't really matter

In [127]:
import time

start = time.time()

outcomes = 0
for _ in range(100000000):
    outcomes += war_game(deck)

end = time.time()
 
print(f'Total outcomes: {outcomes}')
print(f'Total time: {end - start}')

Total outcomes: 1
Total time: 162.93903803825378


### Try 1 billion iterations.

- only had 1/100 million in earlier simulation. seems too rare. 

In [128]:
start = time.time()

outcomes = 0
for _ in range(1000000000):
    outcomes += war_game(deck)

end = time.time()
 
print(f'Total outcomes: {outcomes}')
print(f'Total time: {end - start}')

Total outcomes: 8
Total time: 1593.402357339859
